In [1]:
% load_ext autoreload
% autoreload 2

UsageError: Line magic function `%` not found.


In [62]:
import numpy as np
from utils.depth_operations_functionnal import get_coords_2d
import tensorflow as tf
from tensorflow import keras as ks, float32
from keras import backend as K


def duplicate_constant_along_batch_size(tensor_to_get_batch_size, constant):
    return tf.fill(tf.shape(tensor_to_get_batch_size), 1)

@tf.function
def get_coords_2d_former(map, camera):
    """ Creates a grid containing pixel coordinates normalized by the camera focal length """

    b, h, w, c = map.get_shape().as_list()
    h_range = tf.range(0., h, 1.0, dtype=tf.float32) + 0.5
    w_range = tf.range(0., w, 1.0, dtype=tf.float32) + 0.5
    grid_x, grid_y = tf.meshgrid(w_range, h_range)
    mesh = tf.reshape(tf.stack([grid_x, grid_y], axis=2), [1, h, w, 2]) - tf.reshape(camera["c"], [b, 1, 1, 2])

    divide = tf.divide(mesh, tf.reshape(camera["f"], [b, 1, 1, 2]))

    coords_2d = tf.concat([divide, tf.ones([b,h,w,1])], axis=-1)
    tf.print("concat", coords_2d[0, 0, 0])
    coords_2d = tf.expand_dims(coords_2d, -1)
    return coords_2d, mesh

@tf.function
def get_coords_2d(map, camera):
    """ Creates a grid containing pixel coordinates normalized by the camera focal length """
    # Modif test
    tf.print("get_coords_2d_input", map.shape)
    h, w = map.get_shape().as_list()[1:3]
    h_range = tf.range(0., h, 1.0, dtype=tf.float32) + 0.5
    w_range = tf.range(0., w, 1.0, dtype=tf.float32) + 0.5

    grid_x, grid_y = tf.meshgrid(w_range, h_range)
    mesh = tf.reshape(tf.stack([grid_x, grid_y], axis=2), [1, h, w, 2])

    new_cam_c = ks.layers.Reshape((1, 1, 2,), )(camera["c"])
    new_cam_f = ks.layers.Reshape((1, 1, 2,), )(camera["f"])

    mesh = mesh - new_cam_c

    divide = tf.divide(mesh, new_cam_f)

    # ones_ = ks.layers.Reshape((h, w, 1,), )(tf.ones([1, h, w, 1]))
    myarr = np.ones((1, h, w, 1)).astype('float32')
    myconst = tf.convert_to_tensor(myarr)
    def repeat_const(tensor, myconst):
        shapes = tf.shape(tensor)
        return tf.repeat(myconst, shapes[0], axis=0)

    ones_ = tf.keras.layers.Lambda(lambda x: repeat_const(x, myconst))(map)
    tf.print("ones_", ones_.shape)
    coords_2d = ks.layers.Concatenate()([divide, ones_])
    tf.print("concat", coords_2d[0, 0, 0])

    coords_2d = tf.expand_dims(coords_2d, -1)
    return coords_2d, mesh


map_ = ks.Input(shape=(6, 6, 192,), dtype=float32)  # data image
camera = {"f": ks.Input(shape=(1, 2,), dtype=float32),
          "c": ks.Input(shape=(1, 2,), dtype=float32)}

depth_curr_l_function = lambda x: get_coords_2d(x[0], x[1])
depth_curr_l = ks.layers.Lambda(depth_curr_l_function)((map_, camera))

model_full = ks.Model(inputs=[map_, camera],
                      outputs=depth_curr_l)

b=1
map_test_1 = tf.random.uniform([b] + map_.shape[1:])
camera_test_1 = {"f":  tf.random.uniform([b] + camera["f"].shape[1:]),
          "c":  tf.random.uniform([b] + camera["c"].shape[1:])}

out_put_test_1,_ = model_full((map_test_1,camera_test_1))
print("---")
print()
out_put_test_1_former,_ = get_coords_2d_former(map_test_1,camera_test_1)
print("---")
print()
print()
print("map_test_1", map_test_1.shape)
print("out_put_test_1", out_put_test_1.shape, map_test_1.shape)
print("out_put_test_1_former", out_put_test_1_former.shape, map_test_1.shape)
print(out_put_test_1[0][0][0])
print(out_put_test_1_former[0][0][0])
print()
b=50
map_test_1 = tf.random.uniform([b] + map_.shape[1:])
camera_test_1 = {"f":  tf.random.uniform([b] + camera["f"].shape[1:]),
          "c":  tf.random.uniform([b] + camera["c"].shape[1:])}

out_put_test_1,_ = model_full((map_test_1,camera_test_1))
out_put_test_1_former,_ = get_coords_2d_former(map_test_1,camera_test_1)
print()
print("out_put_test_1", out_put_test_1.shape, map_test_1.shape)
print("out_put_test_1_former", out_put_test_1_former.shape, map_test_1.shape)
print()


get_coords_2d_input TensorShape([None, 6, 6, 192])
ones_ TensorShape([None, 6, 6, 1])
concat [-0.447941601 -0.00479075033 1]
---

concat [-0.447941601 -0.00479075033 1]
---


map_test_1 (1, 6, 6, 192)
out_put_test_1 (1, 6, 6, 3, 1) (1, 6, 6, 192)
out_put_test_1_former (1, 6, 6, 3, 1) (1, 6, 6, 192)
tf.Tensor(
[[-0.4479416 ]
 [-0.00479075]
 [ 1.        ]], shape=(3, 1), dtype=float32)
tf.Tensor(
[[-0.4479416 ]
 [-0.00479075]
 [ 1.        ]], shape=(3, 1), dtype=float32)

get_coords_2d_input TensorShape([None, 6, 6, 192])
ones_ TensorShape([None, 6, 6, 1])
concat [0.302815229 -0.398105502 1]
concat [0.302815229 -0.398105502 1]

out_put_test_1 (50, 6, 6, 3, 1) (50, 6, 6, 192)
out_put_test_1_former (50, 6, 6, 3, 1) (50, 6, 6, 192)

